In [ ]:
from socket import *
import numpy as np
import matplotlib.pyplot as plt
fig,ax=plt.subplots(figsize=(8,8))
serverPort = 12000
serverSocket = socket(AF_INET, SOCK_DGRAM)
serverSocket.bind(('192.168.1.108', serverPort))
print ("The server is ready to receive")
class colorrange:
    def __init__(self,low,up,inout):
        self.up=up
        self.low=low
        self.inout=inout
def bgrm(img,cvar):
    progress=0
    cont=np.array(img)
    if len(img[0][0])!=4:
        cont=cont/255
        conta=np.ones((len(img),len(img[0]),1))
        cont=np.concatenate((cont,conta),axis=2)
    for x in range(len(cont)):
        for y in range(len(cont[0])):
            flag=1
            for i in range(3):
                if(cvar[i].inout==0):
                    if not((cvar[i].low/255<=cont[x][y][i] and cont[x][y][i]<=cvar[i].up/255)):
                        flag=0
                else:
                    if not((cont[x][y][i]<=cvar[i].low/255 or cvar[i].up/255<=cont[x][y][i])):
                        flag=0
            if flag:
                cont[x][y][3]=0
                cont[x][y][0:3]=[0,0,1]
        if (x*100//len(cont)!=(progress//len(cont))):
            pass
            #print(x*100//len(cont)+1,"/100")
        progress+=1
    return cont
def receiveimg(serverSocket):
    header, clientAddress=serverSocket.recvfrom(2048)
    header=header.decode().split()
    print(header)
    height=int(header[0])
    width=int(header[1])
    cvar=[]
    for i in range(3):
        cvar.append(colorrange(int(header[i*3+2]),int(header[i*3+3]),bool(int(header[i*3+4]))))
    for i in range(3):
        print("low=%d"%cvar[i].low,"up=%d"%cvar[i].up,"inout="+("out" if cvar[i].inout else "in"))
    img=np.zeros((height, width,3),dtype=np.uint8)
    for y in range(height):
        for x in range(width):
            pix,clientAddress=serverSocket.recvfrom(2048)
            pix=pix.decode()
            img[y][x][:]=[ord(pix[0]),ord(pix[1]),ord(pix[2])]
    return img,cvar
def sendimg(serverSocket,clientAddress,img):
    height=len(img)
    width=len(img[0])
    header=str(height)+" "+str(width)
    serverSocket.sendto(header.encode(),clientAddress)
    for y in range(height):
        for x in range(width):
            serverSocket.sendto((chr(int(img[y][x][0]*255))+chr(int(img[y][x][1]*255))+chr(int(img[y][x][2]*255))+chr(int(img[y][x][3]*255))).encode(),clientAddress)
while True:
    message, clientAddress = serverSocket.recvfrom(2048)
    if(message.decode()=="start"):
        img,cvar=receiveimg(serverSocket)
        img=bgrm(img,cvar)
        sendimg(serverSocket,clientAddress,img)
        #plt.imsave("receive.jpg",img)
        
        print("finish")


The server is ready to receive
['128', '160', '2', '2', '2', '2', '2', '2', '2', '2', '2']
low=2 up=2 inout=out
low=2 up=2 inout=out
low=2 up=2 inout=out


In [1]:
from socket import *
serverPort = 12000
serverSocket = socket(AF_INET, SOCK_DGRAM)
serverSocket.bind(('192.168.1.108', serverPort))
print ("The server is ready to receive")
while True:
    message, clientAddress = serverSocket.recvfrom(2048)
    print(message.decode())

The server is ready to receive


TypeError: ord() expected a character, but string of length 3 found